<a href="https://colab.research.google.com/github/AdityaJ9801/TRYONDIFFUSION1.0/blob/main/torch_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Build the Neural Network
Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

##Get Device for Training
We want to be able to train our model on an accelerator such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f'using {device} device')

using cpu device


##Define the Class
We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [3]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

# We create an instance of NeuraNetwork, and mopve it to the device, andd print its structure.

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [4]:
X = torch.rand(1,28,28,device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([5])


##Model Layers
Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [5]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


##nn.Flatten
We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [6]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


##nn.Linear
The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [7]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


##nn.ReLU
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.



In [8]:
print(f'before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

before ReLU: tensor([[ 0.0494, -0.2563, -0.3326,  0.1467, -0.4868,  0.0730,  0.0139, -0.2492,
         -0.3228,  0.2118, -0.0789,  0.3837,  0.3424, -0.2965, -0.2820, -0.1947,
          0.0470,  0.2968,  0.5053, -0.0667],
        [ 0.1893, -0.0935, -0.4376, -0.2071, -0.1114,  0.3443, -0.0586,  0.3530,
          0.2095,  0.0214, -0.2583,  0.6581,  0.1443, -0.1174, -0.2480, -0.2480,
         -0.4350,  0.2259,  0.0049, -0.3329],
        [ 0.0561, -0.0275, -0.8386, -0.1007, -0.3321, -0.0685, -0.0230, -0.0517,
         -0.1219, -0.1649, -0.2662,  0.5942,  0.4658, -0.4891, -0.4444, -0.1139,
          0.1056,  0.3455,  0.3145, -0.3464]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0494, 0.0000, 0.0000, 0.1467, 0.0000, 0.0730, 0.0139, 0.0000, 0.0000,
         0.2118, 0.0000, 0.3837, 0.3424, 0.0000, 0.0000, 0.0000, 0.0470, 0.2968,
         0.5053, 0.0000],
        [0.1893, 0.0000, 0.0000, 0.0000, 0.0000, 0.3443, 0.0000, 0.3530, 0.2095,
         0.0214, 0.0000, 0.6581, 0.1443, 0.0000, 0.00

##nn.Sequential
nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

##nn.Softmax
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

##Model Parameters
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [12]:
print(f'Model sricture: {model}\n\n')

for name, param in model.named_parameters():
  print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n')


Model sricture: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0016,  0.0078, -0.0197,  ..., -0.0024,  0.0131, -0.0193],
        [-0.0011,  0.0152, -0.0032,  ..., -0.0246, -0.0165, -0.0041]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0136, -0.0024], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0093, -0.0071,  0.0390,  ...,  0.0434,  0.0046,  0.0017],
        [-0.0140,  0.0323,  0.0256,  ...,  0.0213, -0.0021, -0.0375]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size